In [1]:
import pandas as pd
import numpy as np

Я заниюмаюсь управлением товародвижения в федеральной сети ювелирных магазинов (900 магазинов). Отвечаю за изделия из золота. В данной работе хочу применить машинное обучение для:

- прогноза продаж ассортимента определенного типа для каждого торгового объекта(магазина) в отдельности
- выявление главных признаков торгового объекта(магазина), котороые влияют на продажи ассортимента определенного типа

Делать это буду на примере мусульманских подвесов.
Для анализа буду использовать данные по продажам мусульманских подвесов из серебра, т.к. они присутствуют почти в каждом магазине.
Практическая польза

In [4]:
# Загружаю датасет с продажами серебра и золота

mus_sales = pd.read_excel(
    r'C:\нетология\muslim.xlsx', sheet_name='олап', skiprows=10, usecols=[0,2,3], names=['id','585_msale','925_msale'])

# Убираем пустые значения после 'Общий итог'
lst = mus_sales['id'].tolist()
mus_sales = mus_sales.drop(axis=0,index=np.arange(lst.index('Общий итог'),len(lst)))
mus_sales.tail()

,id,585_msale,925_msale
893,92,0.0,1.0
894,925,0.0,2.0
895,955,1.0,8.0
896,96,7.0,13.0
897,99,12.0,14.0


In [5]:
mus_stock = pd.read_excel(
    r'C:\нетология\muslim.xlsx', sheet_name='олап', skiprows=10, usecols=[7,9,10],names=['id','585_mstock','925_mstock'])

# Убираем пустые значения
lst = mus_stock['id'].tolist()
mus_stock = mus_stock.drop(axis=0,index=np.arange(lst.index('Общий итог'),len(lst)))

# Убираем магазины с нулевыми остатками
mus_stock['sum'] = mus_stock['585_mstock'] + mus_stock['925_mstock']
mus_stock = mus_stock[(mus_stock['sum'] > 0)]
del mus_stock['sum']
mus_stock.tail()

,id,585_mstock,925_mstock
1462,92,26.0,8.0
1463,925,1.0,11.0
1464,955,5.0,10.0
1465,96,22.0,14.0
1468,99,27.0,20.0


In [6]:
# Оставляем склады, где есть остатки золото и серебро одновременно

mus_stock_combined = mus_stock[(mus_stock['585_mstock'] > 5)&(mus_stock['925_mstock'] > 5)].drop(axis=1, labels=['585_mstock','925_mstock'])

# Посмотрим корреляцию в продажах золота и серебра
mus_sales_combined = pd.merge(mus_stock_combined, mus_sales, on='id',how='left').dropna(how='any')
mus_sales_combined[['585_msale','925_msale']].corr().round(decimals=1)

,585_msale,925_msale
585_msale,1.0,0.7
925_msale,0.7,1.0


Коэффицент корреляции между золотом и серебром 0,7. Можно использовать данные по серебру

In [7]:
# Посмотрим есть ли разница в распределении продаж по золоту и серебру
mus_sales_combined.describe().round()

,585_msale,925_msale
count,492.0,492.0
mean,10.0,12.0
std,10.0,12.0
min,0.0,0.0
25%,3.0,5.0
50%,7.0,9.0
75%,14.0,15.0
max,57.0,82.0


Вопросы: 
- нужно ли объединять тц и трц?
- разные размерности и порядок исчисления (продажи и емкость магазина в штуках, зарплата в рублях, доли в процентах)


###### Создаем большой датасет для обучения модели со следующей информацией:

+ id - номер магазина его перед обучением я удалю
+ 925_msale - продажи мусульманских подвесов - целевая переменная
+ type - тип размещения объекта (отдельное здание или торговый центр)
+ city - город или населенный пункт, по этому параметру каждому объекту можно установить признак региона
+ shop_size - вместимость объекта, этот параметр разобью на 5 размерных групп
+ work_time - время работы объекта в годах - разобью на две группы до и более 2 лет
+ total_sales - общие продажи в штуках за полгода
+ brand - вывеска на магазине(есть несколько брэндов)
+ population - количество жителей в данном населенном пункте разобью на 5 размерных групп
+ salary - средняя зарплата (distr - в округе,reg - в регионе)
+ mosque - есть или нет в населенном пункте мечеть(по данным из википедии)
+ muslim_per - процент верующих мусульман (distr - в округе,reg - в регионе)
+ rus_per - процент русских по национальности (distr - в округе,reg - в регионе)
+ religion_imp - процент людей, для которых религия играет важную роль в жизни (distr - в округе,reg - в регионе)


###### Откуда брал данные:

+ Численность населения - Оценка численности постоянного населения на 1 января 2018г.Росстат по городам
+ Доля мусульман, процент русских, процент религиозных - Общероссийский опрос МегаФОМ 29 мая – 25 июня 2012 г. по субъектам РФ
+ Доходы населения - Среднемесячная номинальная начисленная заработная плата работников по полному кругу организаций  по субъектам Российской Федерации. Росстат. 2018-2019 г
+ Количество мечетей - википедия статья Ислам в России
+ Данные по долготе/широте/региону получал через GeoPy по городу


Все данные собрал в единую таблицу в EXcel потом загрузил в ноутбук.

Чеченская  и Ингушская республика не участвовала в исследовании МегаФОМ. Поэтому для этих республик инфу пришлось искать отдельно

In [8]:
# Загружаем справочник с характеристиками магазинов и дополнительной информацией по регионам. 
ref_book = pd.read_excel(r'C:\нетология\muslim.xlsx', sheet_name='типы', usecols=list(range(6,27)))
ref_book['id'] = ref_book['id'].astype(str)
ref_book.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903 entries, 0 to 902
Data columns (total 21 columns):
id                    903 non-null object
city                  903 non-null object
type                  903 non-null object
shop_size             900 non-null float64
work_time             903 non-null int64
total_sales           899 non-null float64
stock_month           903 non-null int64
brand                 903 non-null object
population            898 non-null float64
salary_reg            898 non-null float64
salary_district       898 non-null float64
mosque                903 non-null int64
muslim_per_reg        898 non-null float64
muslim_per_distr      898 non-null float64
religion_imp_reg      898 non-null float64
religion_imp_distr    898 non-null float64
rus_per_reg           898 non-null float64
rus_per_distr         898 non-null float64
coord                 898 non-null object
district              898 non-null object
region                898 non-null object
dtype

In [9]:
# Добавляем к продажам подвесов по магазинам всю информацию из справочника

raw_df = pd.merge(mus_sales[['id','925_msale']],ref_book,how='left', on='id')
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 900 entries, 0 to 899
Data columns (total 22 columns):
id                    900 non-null object
925_msale             900 non-null float64
city                  876 non-null object
type                  876 non-null object
shop_size             876 non-null float64
work_time             876 non-null float64
total_sales           876 non-null float64
stock_month           876 non-null float64
brand                 876 non-null object
population            871 non-null float64
salary_reg            871 non-null float64
salary_district       871 non-null float64
mosque                876 non-null float64
muslim_per_reg        871 non-null float64
muslim_per_distr      871 non-null float64
religion_imp_reg      871 non-null float64
religion_imp_distr    871 non-null float64
rus_per_reg           871 non-null float64
rus_per_distr         871 non-null float64
coord                 871 non-null object
district              871 non-null objec

In [10]:
# Округлим размер и уберем маги, которые работают меньше 4 месяцев 
raw_df['shop_size'] = raw_df['shop_size'].round()
raw_df = raw_df[raw_df['stock_month'] > 4]

In [11]:
# Уберем франчайзинговые магазины - там перекос по ассортименту

raw_df = raw_df[raw_df['type']!='франч']
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 845 entries, 0 to 899
Data columns (total 22 columns):
id                    845 non-null object
925_msale             845 non-null float64
city                  845 non-null object
type                  845 non-null object
shop_size             845 non-null float64
work_time             845 non-null float64
total_sales           845 non-null float64
stock_month           845 non-null float64
brand                 845 non-null object
population            845 non-null float64
salary_reg            845 non-null float64
salary_district       845 non-null float64
mosque                845 non-null float64
muslim_per_reg        845 non-null float64
muslim_per_distr      845 non-null float64
religion_imp_reg      845 non-null float64
religion_imp_distr    845 non-null float64
rus_per_reg           845 non-null float64
rus_per_distr         845 non-null float64
coord                 845 non-null object
district              845 non-null objec

In [12]:
raw_df[['coord']]

,coord
0,"(46.349854, 48.031567)"
1,"(50.595299, 36.586933)"
2,"(45.0433245, 41.9690934)"
3,"(60.709217, 28.744051)"
4,"(51.5013775, 46.1233093)"
5,"(43.4883695, 43.610560193588)"
6,"(45.0433245, 41.9690934)"
7,"(44.894272, 37.316887)"
8,"(55.7655302, 64.5632681)"
9,"(54.991375, 73.371529)"


In [46]:

corr_matrix = raw_df.corr().round(decimals=2)
corr_matrix['925_msale'].sort_values(ascending=False)

925_msale             1.00
muslim_per_reg        0.51
total_sales           0.41
religion_imp_reg      0.38
mosque                0.22
muslim_per_distr      0.20
religion_imp_distr    0.18
shop_size             0.17
salary_reg            0.13
population            0.09
stock_month           0.04
salary_district      -0.13
work_time            -0.14
rus_per_distr        -0.25
rus_per_reg          -0.47
Name: 925_msale, dtype: float64

In [13]:
# Разделим датафрейм на два, в одном будут собраны региональные признаки, в другом признаки по округам 
# возможно какой то датасет даст более точный результат
distr_columns = [x for x in raw_df.columns.tolist() if '_reg'not in x]
reg_columns = [x for x in raw_df.columns.tolist() if '_distr'not in x]

['id',
 '925_msale',
 'city',
 'type',
 'shop_size',
 'work_time',
 'total_sales',
 'stock_month',
 'brand',
 'population',
 'salary_district',
 'mosque',
 'muslim_per_distr',
 'religion_imp_distr',
 'rus_per_distr',
 'coord',
 'district',
 'region']

In [ ]:
# категориальные признаки
pd.get_dummies(data_raw[['offer_class_group']], prefix='CATEGORY', drop_first=True).head(10)

In [ ]:
# from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve
# import xgboost